In [1]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

In [2]:
from datetime import datetime

week_ago = datetime(2016, 10, 29)
contribs = ScrapedContribution.objects.filter(
    date__gte=week_ago).order_by('-amount', '-date').filter(amount__gte=2000)
print 'Found %d contribs' % contribs.count()
counter = 1
for contrib in contribs:
    print '%d) %s from %s to %s on %s' % (
        counter,
        locale.currency(contrib.amount, grouping=True),
        contrib.donor,
        contrib.filing_committee.name,
        contrib.date
    )
    counter += 1

Found 245 contribs
1) $300,000.00 from KAISER ALUMINUM to NO ON 732 SPONSORED BY THE ASSN OF WA BUSINESS on 2016-11-04
2) $200,000.00 from REPRESENT US to INTEGRITY WA on 2016-10-31
3) $155,000.00 from EVERY VOICE to INTEGRITY WA on 2016-10-31
4) $150,000.00 from ELDRIDGE SEAN to INTEGRITY WA on 2016-10-31
5) $100,000.00 from THE DURST ORGANIZATION LP to AUDUBON WA on 2016-11-03
6) $100,000.00 from REPRESENT US to INTEGRITY WA on 2016-11-02
7) $95,823.00 from SEATTLE SYMPHONY to CULTURAL ACCESS WASHINGTON on 2016-11-15
8) $85,000.00 from AUDUBON WASHINGTON to CARBON WA on 2016-11-04
9) $78,311.00 from SEATTLE ART MUSEUM to CULTURAL ACCESS WASHINGTON on 2016-11-09
10) $75,000.00 from KESHET INVESTMENTS LP to CARBON WA on 2016-11-04
11) $60,000.00 from KEMPER HOLDINGS to FAMILIES FOR SUSTAINABLE TRANSIT on 2016-11-02
12) $56,000.00 from FAMILIES FOR SUSTAINABLE TRANSIT to NO SOUND TRANSIT 3 on 2016-11-04
13) $55,000.00 from WASHINGTON STATE REPUBLICAN PARTY to WYMAN KIMBERLEY M on 2016-1

In [ ]:
all_recent_contribs = ScrapedContribution.objects.filter(date__gte=week_ago)
donors = all_recent_contribs.values('donor').annotate(sum=Sum('amount')).order_by('-sum')
for donor in donors:
    print '%s gave %s' % (donor['donor'], locale.currency(donor['sum'], grouping=True))
    filtered_contribs = all_recent_contribs.filter(donor=donor['donor']).values(
        'filing_committee__name').annotate(sum=Sum('amount')).order_by('-sum')
    for contrib in filtered_contribs:
        print '%s to %s' % (locale.currency(contrib['sum'], grouping=True), contrib['filing_committee__name'])
    print '---'

KAISER ALUMINUM gave $300,000.00
$300,000.00 to NO ON 732 SPONSORED BY THE ASSN OF WA BUSINESS
---
REPRESENT US gave $300,000.00
$300,000.00 to INTEGRITY WA
---
WASHINGTON STATE REPUBLICAN PARTY gave $212,000.00
$80,000.00 to WYMAN KIMBERLEY M
$60,000.00 to BRYANT WILLIAM L
$30,000.00 to MCLAUGHLIN STEVEN A
$13,000.00 to WALSH JAMES W
$10,000.00 to MILOSCIA MARK A
$5,000.00 to O'BAN STEVEN T
$5,000.00 to MCCLENDON MARTIN V
$5,000.00 to LOW SAMUEL E
$4,000.00 to KRAFT VICKI C
---
EVERY VOICE gave $155,000.00
$155,000.00 to INTEGRITY WA
---
ELDRIDGE SEAN gave $150,000.00
$150,000.00 to INTEGRITY WA
---
AUDUBON WASHINGTON gave $120,000.00
$120,000.00 to CARBON WA
---
THE DURST ORGANIZATION LP gave $100,000.00
$100,000.00 to AUDUBON WA
---
SEATTLE SYMPHONY gave $95,823.00
$95,823.00 to CULTURAL ACCESS WASHINGTON
---
SEATTLE ART MUSEUM gave $78,311.00
$78,311.00 to CULTURAL ACCESS WASHINGTON
---
KESHET INVESTMENTS LP gave $75,000.00
$75,000.00 to CARBON WA
---
KEMPER HOLDINGS gave $61,000.0

In [ ]:
all_recent_contribs = ScrapedContribution.objects.filter(date__gte=week_ago)
recipients = all_recent_contribs.values('filing_committee__name').annotate(sum=Sum('amount')).order_by('-sum')
for cmte in recipients:
    print '%s raised %s' % (cmte['filing_committee__name'], locale.currency(cmte['sum'], grouping=True))
    filtered_contribs = all_recent_contribs.filter(filing_committee__name=cmte['filing_committee__name']).values(
        'donor').annotate(sum=Sum('amount')).order_by('-sum')
    for contrib in filtered_contribs:
        print '%s from %s' % (locale.currency(contrib['sum'], grouping=True), contrib['donor'])
    print '---'

In [ ]:
expenses = ScrapedExpense.objects.filter(date__gte=datetime(2016, 10, 1), amount__gte=10000).order_by(
    '-amount', '-date')
print 'Found %d expenses' % expenses.count()
for expense in expenses:
    print '%s to %s from %s on %s' % (
        locale.currency(expense.amount, grouping=True),
        expense.vendor,
        expense.filing_committee.name,
        expense.date
    )